In [92]:
import sqlite3
import pandas as pd
import numpy as np
from API_Key import key
import requests
import time

In [2]:
conn = sqlite3.connect('summoner.db')

# Creating Summoner Table for first API Call

In [ ]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE summoner(

                leagueId TEXT,
                queueType TEXT,
                tier TEXT,
                rank TEXT,
                summonerId TEXT PRIMARY KEY,
                summonerName TEXT,
                leaguepoints INTEGER,
                wins INTEGER,
                losses INTEGER,
                veteran BOOLEAN,
                inactive BOOLEAN,
                freshBlood BOOLEAN,
                hotStreak BOOLEAN) ''')

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

# Setting up loop to obtain all ranked TFT players in Diamond and below

In [ ]:
def player_collector(key):
    leagues = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
    divisions = ['I', 'II', 'III', 'IV']
    summoner = []
    for league in leagues:
        for division in divisions:
            response = requests.get(
            "https://na1.api.riotgames.com/tft/league/v1/entries/{}/{}?api_key={}".format(league, division, key))
            for user in response.json():
                summoner.append(tuple(user.values()))
    return summoner

In [ ]:
players = player_collector(key)

# Updating summoner table 

In [61]:
conn = sqlite3.connect('summoner.db')
c = conn.cursor()
c.executemany('INSERT INTO summoner VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)', players)

conn.commit()
conn.close();

In [ ]:
update_table("summoners", players)

# Creating new table for PUUID

In [87]:
c = conn.cursor()
c.execute('''CREATE TABLE userids(

                id TEXT,
                accountId TEXT,
                puuid TEXT PRIMARY KEY,
                name TEXT,
                profileIconId INTEGER,
                revisionDate INTEGER,
                summonerLevel INTEGER) ''')

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

OperationalError: table userids already exists

In [104]:
def puuid_collector(key):
    conn = sqlite3.connect('summoner.db')
    c = conn.cursor()
    players = c.execute("SELECT summonerName FROM summoner")
    users = [player[0] for player in players]
    summoner = []
    for player in users:
        response = requests.get(
        "https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{}?api_key={}".format(player, key))
        summoner.append(tuple(response.json().values()))
        if len(summoner) % 20 == 0:
            print(summoner[-1])
            print("waiting 20 seconds")
            time.sleep(20)
    return summoner;

In [105]:
puuid = puuid_collector(key)

('vgLkHXPcicOmdJC2dNPp1RA-61ju5klhkMjjjAJNP3lLGQqy', 'YvXppif3GsngiFvLYI1D_-E0c2O80G4SarOlmCvhpOX6CVuizDGr9OIk', 'A4Z3xbRS0ifwqm_anIQgzG9E-SQrn9sh_mKYJ93Ha_BgKHd_stioTTzndC7-udJ8C6C1GpwkADcMmA', 'Crookmass', 4630, 1610293614000, 70)
waiting 20 seconds
('vby8vnoXCswXNQQL3I2b7Wm6hpxMBo0pJQhZAuQb8pMC_8wW', 'KVDonBVoxouw4HGPv3QsyYEcQqjvOyz5GJSAc2jGWICy-I5BK_DZ6zbQ', 'lxtO_hhMPuxdPeIdthtrUGf5XwahNHZ33ttAmfF3nzzidxpn5WsNTPjWdAsLNrb69UAb8Ne9ko0zoQ', 'SleepyGreenTea', 4763, 1610578959000, 79)
waiting 20 seconds
('NG7f3EOM4mh5TH9wYxPLmMKv5t5AczDyoqALRcdY-yKa4u6v', 'GeipHIWZ7W9QcKoIhyfP1XX2q9XLqQs_u2TRWywoZnZOka8Ppl9F-36Z', 'A_i5c01rSlS-70vseO2pcbDaEv9ZsP3s6wbtIVaHwj42HBHfQOcAG8xGK6ogDd3m5xBzOgpwW2RYFw', 'LiquidJoker', 4862, 1610928870300, 71)
waiting 20 seconds
('LnJC39lQTsvi-Is1e14HI_elLFiLnfrjD3hssGPX3B95h93K', '_kmxAs23E-Po4OrmHl_kULE1AYkgC1bCCxKQ3RB5-b9sZecTL0-GFrXJ', 'QgTCxAFzJppe_hspYWsCkxgxWjIkDQ0QphR6ZPOm9lsLTNxNd9U9eR1xKU0vH3rLWbVDjRkZnyYWHA', 'Kamisamoo', 4834, 1610741338000, 28)
wait

('Imh9r8ImBLu-F9nG0Wqx88UFHgHarcipUxKASKTF2OiX6Jta', 'z3Vf6K6FBdyLvYfo2oSPHYzDjlVaFUrExEVQbKG90rgkO_Wpkr7cfgK2', '9i59QOfRBWLCrDTbAAIGmszXd7MNMd_Kbf_gI38VccnLsqXT8vVHzt0ipgrFmBYaXCus2Lds4ql4ig', 'ptarer', 29, 1610424143000, 1)
waiting 20 seconds
('pUYfpiUguTImCfZToyU1zeNXlOeMsCnUM-7RjPx2tB_Jmq0g', 'pmTaUAlva6puLhHZxzKoexrTcw4VTM_tag9Mem4SDs5Bm6o_0eizvp8V', 'IEd-VcKcCczDkRjHj_MMz9wdobRKGIblwSNS7R19T0XKT8-fhkuovThA8g03_JPFiUP57WRf9q8y9g', 'Choke Meee', 23, 1611162243000, 1)
waiting 20 seconds
('_tk-UEAhmhzMtMrcQFX_UmSnwnizMzJj7cQ2eio9g3kOC0Yx', '4FfZHRFdAAhD1UN11-durD7EZFJG7eBR5oQELdeOYYX4IYPCkGBqekku', 'VnbfO1ZI94UtwO3lYz_mzUwtyI0lDGPW3waYsIJF5mBQKbJkyX9eRcOXQVP-v1GZR3hsgxJjXKutIw', 'YinRaven', 3545, 1605975527000, 4)
waiting 20 seconds
('Ag9LLuWUrgc3RfFyerZ65IQ1gYcPulXDPc-7FfQFGcpMrBAt', 'zk0F4KJG5GAIKXyY-RGLTizzW99ATkQ-TtXoBkqbXoIH4u_1YWubj9-D', 'f4LPfEm8yaREixfPcKmyTd-QxAvSHBHRmW1nk6yD5N2LaupcGpqDEHGksh7FADv3LLjAEwD9TYP-Bg', 'Kariand', 3543, 1609107949000, 4)
waiting 20 seconds
('NSk

('9_OBNksQbywUCcpmu0kEUtOUt5IGNFDfcNZYo_enub4HQsLU', '4WDDxpdR8bWiVBmgCKLdtaQc71oriOCUuOV0ihUDXBpRuXA', 'nboifZBEXbvTDaFHsPd9fyUkmcrKPy42wIoLeTZv1nutim0ARfbIo-M6wRxdfpGzaFYelCFiJvTBxg', 'Adventurer Finn6', 3544, 1610952704000, 33)
waiting 20 seconds
('UFreqo28sUL00KTSApeIbSL3BESI4-k_1iHmPjY9N8h5diI9', 'vSWYfSH_t9JLbOY3OXyNFLnLjeBbxuj0RwUqj56QKMK7mh9SlLIUe8sM', 'CFRTVtYQPgs5SOtrv2Sz6WZwLkg-eKXqFpU_LtcbmPy_O_z1oRJH9NNzIwLfzSgygLgEKqGsmC6rkQ', 'Apex67', 4832, 1609848976000, 31)
waiting 20 seconds
('d7uXp2rhQQpZ2MyahcPsXBRlZwbJCJ6rrndVgQYBoW6yeu4m', 'SDRIndIBt0iaYPcyr9jRzbR-gLZeO-ztVbt_kQ63oRqqNywER-F_79xW', 'oaA71BPmgkWxvJs4Utvgico7Wni-bvsC3UeU4HxMoB1D-nH0yQqQtMrUUdLD2zJ5eFwYRDZfr2r8GA', 'DetectiveN', 4407, 1611206165000, 242)
waiting 20 seconds
('MKi3zKaxgaTeoXW0WEbjRt0xAuYRZaPCzK8GiOjGZn2oDXlW', '9feVETl1ApLGn-Eayl66tNhvTWavaQic1cOCn7zPtSnPKBHQCg89F-Gv', '2RcQ-4oX3Q56juKAh6OWEw9YH1IXkaPiBPPTWH4tk1teTpE6hwk6cI85YWBotevokaWeyKV0oW2v6w', 'CantResistTriss', 3458, 1603798185000, 108)
waiting

('Ii9_zrTMQrMIGkT7Hvs1B9LRtllA2V5HJwN8giOn2mLruB2K', 'zx34uz8F_DHG_ZXv6G_0sbjZdJKDII4aRWQUQJul5pn2Dn26bfvt4lAO', 'xm1bu8SEGw7MiQKMOABd20IT4wNcXPgKV_vSIEODQrey4DBhAVektU4jIZaC_SGxUbjD7NtwZD2K6w', 'NotReallySora', 4657, 1611260244426, 252)
waiting 20 seconds
('aUPXeP_dVut3u36cPff-P5z9ceKagdOLBMMF0pL8UM17UU3z', '3WEZuAJ728-6aNVehR4RT3B_LxKPx4Bfk1i4i4-9DmG6adSAZzNNMB1g', 'TO4T2a-qwu-8WhWep-EaeDv3hpBOor_LBsVdkC0u7wxEI2NIGdnyxqTp4r9wuzSb8KlJbA0G41ud3Q', 'SpapezOP', 4243, 1609408062000, 71)
waiting 20 seconds
('ZIEagQco51vLZThnXZhdJqmztuaiK43qOnYrBsq8K868BZAe', 'f7iTsZGIJzQfbdVNSv_TGqVsbIlHHVKcwLJSMAYMpF0yPKZt9Iof9q5K', 'K_cDGOOnTVrGAu7CH28rNyZ47HpzRFQtGWVI-WSsAffIqc4ie5NlL6JwzWd9fDxi_du3Pv6svXiGiA', 'ichikochan', 3542, 1600667960000, 6)
waiting 20 seconds
('1glY9Jo_J6_3iwpUKwDy2sKaqVKC0xE9ZCcan23cSo8Ci7O8', '5fACZbSj_JyQCsbqjwbdvqx7g0WXI5_WX0SpLUqPHN-mA94jwP8DpgHk', 'dPAzfOK8S4ejtcjLtKLExJBjQtgvQ7-FRWSw7VBM7cZvdo9_UwSnHE1euQd1fVs6kaHY18WXRDj7HA', 'Oshino Oug1', 4793, 1611278490741, 202)
wait

('gahOyH2b3PctvbDw3v_OVYBAUiwJsWf7Xr6DhTFWgbaV44AR', 'JnWBtHFhn5teBXqYLqcQGS1TBwOr7d-tYFL7_Ot4dgvb9w3L7h7jQO9d', 'uZiD5_Ih39IkoobrOh6tL0mD1EcDU_gAL2EnwqmZkNGhrapRdzBzR_Y2pIhaLc62PTmMLJB6yKKSzg', 'ProNinjaTeaY', 4409, 1610959043000, 188)
waiting 20 seconds
('nChhaP1yRm68l3QDDJl35f5h-MlX4LwTCSlQ4APce_ZmchLh', 'eCLqA_tU9H7YDsdJpJfrb6kqHMsp7Ao9kQ4OJ0Oz015-Zyoj0gDfIY45', 'Q_2AriFTmYdmbTwtH-bbzxe0Xr2gn8M4-IHTQofWe7OommE-SI-iPte1jITSaVLtV8WKJ8pI4KxTlA', 'LonelyZee', 22, 1611124673000, 119)
waiting 20 seconds
('dusXVWay0nXSXw_BDbCJKBX82U-CW1NR6oLD1WlEdWbXI2fK', '5iGS6tvI-Z3ZPQbEhiTB9P9iOEmEMx7sr6GH7GTRP2iyhRO1dV1ivB3x', 'qshGFaeD6tDVW23_PjDouZca1BAo99A5nKCO1Oe9LZAvSgergnRj9pZPVNmeVGNOpQ3Qtq01yIi1QA', 'BeNiceToMeOrDie', 1640, 1611276238743, 285)
waiting 20 seconds
({'message': 'Data not found - summoner not found', 'status_code': 404},)
waiting 20 seconds
('C7ooSrBzebvyqXlHmvGm9Szz-etjCWp6DdydIiK1q1qBXsUi', 'fbyRjd5f9szgC3cN8ADWKBN2J2TU9iBw3d7X1SSlewb5yMvwyTcD4IgH', 'IAcytJLdNUokcfin6Tq7SZ7CKVk

('mRW_nZRHmc1Qszn4M0wPefO1-9EYB61J9b8ZqjlXlaogsJkH', 'vPkxa_Cv9M9RsEYzuLwzFJ2DShcrUaVuF_SVymKyaO7fNWTSBTRMU0wD', 'd3urrNeSHPmWaer53Nuc-pVPYCW3dufAlezl2Qu3LmuNrZt_68S7ImmCWuqjHtr--PCrC80SedhvDA', 'Piewashere420', 1427, 1611277617000, 98)
waiting 20 seconds
('ft1AqAkgF6tDzRT0Tm34EdBEc8TvPwv_OX9NbQyCXmSqf0_9', '1dStuuelSzeARIUgmLWsAoKbfQdEBcN9DoeA3xDeDNJvzKbw2n3QoWZi', 'o_15fJnWo-iWNCl0cZvcNubHaHFjRVdUHLu_mUksWvd10WAsuuS5XLdC-UF0KvHt5FrGmKDmg2xhUA', 'IzumiSagiri Suki', 4653, 1611249864000, 248)
waiting 20 seconds
('ef4gl1xmhzGukgglqeAYAl7olDIAVZ1uMGTghnyoH9CV7yny', 'LbakB3fbEnoxJBjJlHjJh4KS9DUPcuLtX9ycsqblrFWDldqMccV8jClF', 'gBmSM4h30m-fOiEwrx-_wOIOocLXCGE3oZTk1LidkwNYjMaRmlC4F6GFhb9iblJYBkn7IxHsTGnL2w', 'Fish4Hoesdotcom', 1296, 1611265419237, 28)
waiting 20 seconds
('gAYk0EiUUbXxizC3TuFWYZl0Zx_gikMANL4U1fIP7Q_lg9Dr', 'n2Z5VZDY0KamSt8kdkdmn3JBL0dt_AOZp1D6jzzvv-CTj4ONiAXM_679', 'GqldsLqOh-Hdge38rjLqaw9i_G5JKMQjoh0IOQzISXBLGtWsaK2VuH9vTmLD6R93JiTx64G0V4VvgA', 'Carry me yoyo', 4363, 16106835

('NYgYXHV_gnxfzKzZPrmqXZyGNQAk9ZdQoCBRBULZ8K4E4hNU', 'H7_GC5380gRW-yuvjIOKuTOfGv7PI0vu2DAu9M0GytRtHnPRk3EXNj9r', 'NTM4OW7-1i28af1qziPP4uSO7svHMd70-TkwXCIy4GjuQV8tDno8V2JtVouK8zBiiCq3Oms4MNPWOw', 'yytzdlh', 3478, 1610780259000, 173)
waiting 20 seconds
('hgyWHe8wFmGsr5DmCH5FvZw1DRdxLt3kLUY0KKgocuw13Gdc', '7mGSkMzN358d5sly96m6acapMRYHgYSLAGk9s0JfUOtExNCsYnknurtF', '5Kg8iBz7rh_uxjSNSxaeZspzhxK-gT-X9KkYcnzuJRwsFMmsgBckSE9ZXMyzXxCVFbyF6GVWo3glHg', 'ppppeter', 3550, 1607666957000, 21)
waiting 20 seconds
('ojM6VDgiM0ArUNEmZ_9u3vGrc5Yk5O8Jz3I5gOG8J9QnJW8u', 'amuIdRvlB_98ey0wr-s_lj-W9fYV3wJg2qrcYgNzn1l03cRrVGuPMAQ7', 'WqzHqGGsyL43vwcLH9Zh1mNLahZ7oEAK8aeBGLy7FdGf7GPohfskDdoJpnotQAdfG_hIm6qXFhGbvA', 'NhaGiaKim', 4777, 1610938844000, 84)
waiting 20 seconds
('dOiY3NnhJZyTE0uLp-2twpc9zXxLzSrLxzTK2EghObyL5GFQ', 'XDuTHRKcAoqeAeR3lj5kW2qMA-KUd6GkOfFN5nNU6U3WNpFjvifz68um', 'wo1NGcgGEXu0sMnIYNNkEUYYqKP8z9Y7IV_BPMiduNdsBO2D-SDIGt6bafRoVSaQFMosn4Emq90OZQ', 'AddChickpeas', 23, 1601963255000, 11)
waiting 20 s

('EQr_O67aQPnUH3NgrqXjscuCYkypZgrHowupevNB9or8ezN1', 'U-yRG4CcGi_fxSNVVlC_g4m0Agz4frQFYJLlVt_904xEWoABQUOFhRsv', '6LQlk3QC-0zZAxB_KkM4bd6fhB25Rh72E-B-7SOhu-54zgWOEQWVgJ3n8jISGjp2onxrhnQDjLuFTg', 'Austeyen', 23, 1611277336941, 12)
waiting 20 seconds
('IkZMTDLqQd_0hACxfqarsPIfsHXs4LadsDpf_CNHUq8qBz6G', 'TzFO0ehm6jcY6FXziE9aNcls5HyBLQJVxH6u5A0anYhPlJWDHu_WWaak', 'U2DZdVs86IpbF9GxDyL8wqi49BdsCCkIoG1dHAnWSq77_TixTK2PKXmMHzQUs4lZ081jnvoBPkHS7Q', 'Suukey', 2076, 1603767171000, 92)
waiting 20 seconds
('YoEfGSZVBF7W27lZ13MN5Ouvc2ePYMG7yYXoGVEevGs-4KaR', 'qG3Xyf57gnBKQK1S_XFi7uzoegJKjWW6EThihuc1T9XozeDKA8k-ryTm', 'WTc8lnk4cMRS4v6UOqmBQ6lOPBZBzgIJ51on1MEdBkm8iywF9mdMA2F53n8mG8mt321VL8_udXCtTQ', 'fiddlethicks', 3478, 1610239766000, 31)
waiting 20 seconds
('L3HjdZgkwweLTg21c-eXFw-bwfcULnisepwpwnMYtQ469WmU', 'pZQ0NIkLux4zn7JPpDjbYxrnuHJdujdJ9mAUXbNBfnzgV5oHUMLJcv9v', 'amImeuhCtFeQDzP-iSbvmrRcLv24reKchPqFlo3DzpFx4YA0WZO1x9lTK58tqL2IZ1GklQZkC9nGPA', 'SirDur', 17, 1610806952000, 151)
waiting 20 seconds

# Some summoner names were not available in this search so filtered them out

In [130]:
filtered_puuid = filter(lambda i:not(type(i) is dict), puuid)

In [138]:
filtered_puuid

In [143]:
puuid_vals = [val for val in filtered_puuid]

# Adding data to new table in DB

In [142]:
conn = sqlite3.connect('summoner.db')
c = conn.cursor()
c.executemany('INSERT INTO userids VALUES(?,?,?,?,?,?,?)', puuid_vals)

conn.commit()
conn.close();